<a href="https://colab.research.google.com/github/Ototex2811/Ototex2811/blob/main/Extract_climate_from_ERA5_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ERA5-Land Daily Data Extractor for Unique Points from GeoTIFFs (Batch Processing)
# Extracts daily data for deduplicated points in year batches to avoid memory issues

# @title ### ⚙️ Setup
from google.colab import drive
import xarray as xr
import rioxarray
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import glob
import re
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive')

# Install rioxarray
!pip install rioxarray

# @title ### 📝 Configuration
OUTPUT_DIR = "/content/drive/My Drive/GEE_ERA5_Output"
POINTS_CSV = "/content/drive/My Drive/230.csv"
OUTPUT_CSV = "/content/drive/My Drive/GEE_ERA5_Output/germany_points_1960-2020.csv"
INTERMEDIATE_DIR = "/content/drive/My Drive/GEE_ERA5_Output/intermediate"

VARIABLES = [
    'temperature_2m', 'dewpoint_2m', 'pressure', 'precipitation', 'soil_moisture',
    'wind_speed', 'wind_direction', 'vpd', 'rin', 'rli', 'le'
]

# Batch processing configuration
YEAR_INTERVAL = 5  # Number of years per batch (adjustable, e.g., 1, 5, 10)
START_YEAR = 1960
END_YEAR = 2020

# @title ### 🔍 Verify Inputs
# Check points CSV
if os.path.exists(POINTS_CSV):
    points = pd.read_csv(POINTS_CSV)
    print(f"Points CSV: {POINTS_CSV}")
    print(f"Columns: {points.columns.tolist()}")
    print(f"Total points: {len(points)}")
    print(points.head())

    # Deduplicate points based on ID, LAT, LON
    points = points.rename(columns={'ID': 'point_id', 'LAT': 'lat', 'LON': 'lon'})
    unique_points = points[['point_id', 'lat', 'lon']].drop_duplicates()
    print(f"Unique points after deduplication: {len(unique_points)}")
    print(unique_points.head())

    # Use unique points
    points = unique_points
else:
    print(f"Points CSV not found: {POINTS_CSV}")
    raise FileNotFoundError("Please provide the points CSV.")

# Check GeoTIFFs
tiff_files = glob.glob(f"{OUTPUT_DIR}/germany_climate_data_daily_*.tif")
if tiff_files:
    print(f"Found {len(tiff_files)} GeoTIFFs in {OUTPUT_DIR}")
    # Group by year
    year_files = {}
    for tiff in tiff_files:
        year = re.search(r'daily_(\d{4})\.tif', tiff).group(1)
        if year not in year_files:
            year_files[year] = []
        year_files[year].append(tiff)

    for year in sorted(year_files):
        print(f"Year {year}: {len(year_files[year])} GeoTIFF(s)")

    # Inspect sample GeoTIFF
    sample_tiff = year_files[list(year_files.keys())[0]][0]
    ds = rioxarray.open_rasterio(sample_tiff)
    print(f"Sample GeoTIFF: {sample_tiff}")
    print(f"Dimensions: {ds.dims}")
    print(f"Band count: {ds.sizes['band']}")
    print(f"Sample band names (first 20): {ds.coords['band'][:20].values}")
    ds.close()
else:
    print(f"No GeoTIFFs found in {OUTPUT_DIR}")
    print("Listing directory contents:")
    !ls -l "{OUTPUT_DIR}"
    print("Listing parent directory:")
    !ls -l "/content/drive/My Drive/"
    raise FileNotFoundError("Please verify GeoTIFFs exist in the correct directory.")

# @title ### 📍 Extract Data in Batches
# Create intermediate directory
os.makedirs(INTERMEDIATE_DIR, exist_ok=True)

# Generate year batches
year_batches = [(start, min(start + YEAR_INTERVAL - 1, END_YEAR))
                for start in range(START_YEAR, END_YEAR + 1, YEAR_INTERVAL)]
print(f"Year batches: {year_batches}")

results = []
for batch_start, batch_end in tqdm(year_batches, desc="Processing batches"):
    batch_results = []
    batch_years = [y for y in year_files if int(y) >= batch_start and int(y) <= batch_end]

    for year in tqdm(batch_years, desc=f"Years {batch_start}-{batch_end}", leave=False):
        tiff = year_files[year][0]  # Single GeoTIFF per year
        ds = rioxarray.open_rasterio(tiff)
        ds = ds.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time'})

        # Calculate days based on leap year
        year_int = int(year)
        is_leap = (year_int % 4 == 0 and year_int % 100 != 0) or (year_int % 400 == 0)
        days_in_year = 366 if is_leap else 365
        days = pd.date_range(f'{year}-01-01', f'{year}-12-31', freq='D')

        # Verify band count
        expected_bands = days_in_year * len(VARIABLES)
        actual_bands = ds.sizes['time']
        if actual_bands != expected_bands:
            print(f"Warning: Year {year} has {actual_bands} bands, expected {expected_bands}")
            days_per_year = actual_bands // len(VARIABLES)
            days = days[:days_per_year]

        # Split bands into variables
        var_data = {}
        bands_per_day = len(VARIABLES)
        for i, var in enumerate(VARIABLES):
            var_data[var] = ds.isel(time=slice(i, None, bands_per_day))
            var_data[var] = var_data[var].assign_coords(time=days)

        # Create dataset
        ds_year = xr.Dataset(
            {var: (['time', 'latitude', 'longitude'], var_data[var].values) for var in VARIABLES},
            coords={'time': days, 'latitude': ds.latitude, 'longitude': ds.longitude}
        )

        # Extract data for points
        for _, row in points.iterrows():
            point_data = ds_year.sel(
                latitude=row['lat'],
                longitude=row['lon'],
                method='nearest'
            )
            point_df = point_data.to_dataframe().reset_index()
            point_df['point_id'] = row['point_id']
            point_df['lat'] = row['lat']
            point_df['lon'] = row['lon']
            batch_results.append(point_df)

        # Free memory
        ds.close()
        ds_year.close()
        del ds, ds_year, var_data

    # Save intermediate CSV for the batch
    if batch_results:
        batch_df = pd.concat(batch_results, ignore_index=True)
        columns = ['point_id', 'lat', 'lon', 'time'] + VARIABLES
        batch_df = batch_df[columns]
        batch_csv = f"{INTERMEDIATE_DIR}/germany_points_{batch_start}-{batch_end}.csv"
        batch_df.to_csv(batch_csv, index=False)
        print(f"Saved intermediate CSV: {batch_csv}, Rows: {len(batch_df)}")
        results.append(batch_df)
        del batch_df, batch_results

# Combine all intermediate CSVs
if results:
    final_df = pd.concat(results, ignore_index=True)
    final_df = final_df[columns]
    os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
    final_df.to_csv(OUTPUT_CSV, index=False)
    print(f"Saved final CSV: {OUTPUT_CSV}")
    print(f"Rows: {len(final_df)}")
    print(final_df.head())
else:
    print("No data processed. Check intermediate CSVs or input files.")

# @title ### 📊 Verify Output
if os.path.exists(OUTPUT_CSV):
    output_df = pd.read_csv(OUTPUT_CSV)
    print(f"Output CSV: {OUTPUT_CSV}")
    print(f"Columns: {output_df.columns.tolist()}")
    print(f"Rows: {len(output_df)}")
    print(output_df.head())
else:
    print(f"Output CSV not created: {OUTPUT_CSV}")
    print("Listing intermediate CSVs:")
    !ls -l "{INTERMEDIATE_DIR}"

Mounted at /content/drive
Points CSV: /content/drive/My Drive/230.csv
Columns: ['ID', 'LAT', 'LON']
Total points: 12198
     ID      LAT       LON
0  3213  48.0833   7.63333
1  4641  49.4167   6.91667
2  2241  51.1333   9.01667
3  1980  49.7167   8.83333
4  4266  49.0167  11.11670
Unique points after deduplication: 229
   point_id      lat       lon
0      3213  48.0833   7.63333
1      4641  49.4167   6.91667
2      2241  51.1333   9.01667
3      1980  49.7167   8.83333
4      4266  49.0167  11.11670
Found 61 GeoTIFFs in /content/drive/My Drive/GEE_ERA5_Output
Year 1960: 1 GeoTIFF(s)
Year 1961: 1 GeoTIFF(s)
Year 1962: 1 GeoTIFF(s)
Year 1963: 1 GeoTIFF(s)
Year 1964: 1 GeoTIFF(s)
Year 1965: 1 GeoTIFF(s)
Year 1966: 1 GeoTIFF(s)
Year 1967: 1 GeoTIFF(s)
Year 1968: 1 GeoTIFF(s)
Year 1969: 1 GeoTIFF(s)
Year 1970: 1 GeoTIFF(s)
Year 1971: 1 GeoTIFF(s)
Year 1972: 1 GeoTIFF(s)
Year 1973: 1 GeoTIFF(s)
Year 1974: 1 GeoTIFF(s)
Year 1975: 1 GeoTIFF(s)
Year 1976: 1 GeoTIFF(s)
Year 1977: 1 GeoTIFF(s)


Processing batches:   0%|          | 0/13 [00:00<?, ?it/s]

Years 1960-1964:   0%|          | 0/5 [00:00<?, ?it/s]

Saved intermediate CSV: /content/drive/My Drive/GEE_ERA5_Output/intermediate/germany_points_1960-1964.csv, Rows: 418383


Years 1965-1969:   0%|          | 0/5 [00:00<?, ?it/s]

...

Saved intermediate CSV: /content/drive/My Drive/GEE_ERA5_Output/intermediate/germany_points_2010-2014.csv, Rows: 418154


Years 2015-2019:   0%|          | 0/5 [00:00<?, ?it/s]

Saved intermediate CSV: /content/drive/My Drive/GEE_ERA5_Output/intermediate/germany_points_2015-2019.csv, Rows: 418154


Years 2020-2020:   0%|          | 0/1 [00:00<?, ?it/s]

Saved intermediate CSV: /content/drive/My Drive/GEE_ERA5_Output/intermediate/germany_points_2020-2020.csv, Rows: 83814
Saved final CSV: /content/drive/My Drive/GEE_ERA5_Output/germany_points_1960-2020.csv
Rows: 5102349
   point_id      lat      lon       time  temperature_2m  dewpoint_2m  \
0    3213.0  48.0833  7.63333 1962-01-01      273.693233    -0.451600   
1    3213.0  48.0833  7.63333 1962-01-02      271.403597    -3.014711   
2    3213.0  48.0833  7.63333 1962-01-03      265.990917    -9.197239   
3    3213.0  48.0833  7.63333 1962-01-04      272.020940    -2.816962   
4    3213.0  48.0833  7.63333 1962-01-05      273.849033    -1.478681   

     pressure  precipitation  soil_moisture  wind_speed  wind_direction  \
0  986.861193       2.024975       0.440536    1.318640      -81.760762   
1  990.620428       7.737735       0.440770    3.031445      -94.126968   
2  992.183755       0.477300       0.441024    2.399765      -92.304761   
3  995.645692       1.337102       0.44128